# Attentive FP

## Featurizing molecules

In [1]:
!pip install --pre deepchem
!pip install wandb --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 608 kB 5.1 MB/s 
     |████████████████████████████████| 36.8 MB 12.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 5.2 MB/s 
     |████████████████████████████████| 158 kB 53.8 MB/s 
     |████████████████████████████████| 181 kB 52.8 MB/s 
     |████████████████████████████████| 63 kB 1.4 MB/s 
     |████████████████████████████████| 157 kB 59.7 MB/s 
     |████████████████████████████████| 157 kB 52.1 MB/s 
     |████████████████████████████████| 157 kB 56.1 MB/s 
     |████████████████████████████████| 157 kB 63.0 MB/s 
     |████████████████████████████████| 157 kB 42.9 MB/s 
     |████████████████████████████████| 157 kB 53.6 MB/s 
     |████████████████████████████████| 157 kB 53.5 MB/s 
     |████████████████████████████████| 156 kB 52.1

In [2]:
import deepchem as dc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# RDKit
import rdkit
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import PandasTools
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem.Draw import SimilarityMaps

import os
from pathlib import Path
import csv
import math

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [3]:
content_path = Path('/content/drive/MyDrive/chem_data')
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !pip install -qU deepchem wandb

In [4]:
import wandb
wandb.login()

import warnings
warnings.filterwarnings('ignore')

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [5]:
from deepchem.models.wandblogger import WandbLogger

In [6]:
# Create directories
# def create_dir():
""" Create directories featurized, valid, train, test.."""
# create_dir()
gat_dir = content_path/'gat_dir'
if not gat_dir.is_dir():
  gat_dir.mkdir()


# create_dir()
attentiveFP_dir = content_path/'attentiveFP_dir'
if not attentiveFP_dir.is_dir():
  attentiveFP_dir.mkdir()

In [7]:
# Featurize data into dc dataset
def csv_to_feat(filename, feat="attentiveFP"):

  data = pd.read_csv(f"{content_path}/{filename}.csv")
  smiles = data['smiles'].tolist()
  pki = data['pki'].tolist()
  if feat == "attentiveFP":
    featurizer = dc.feat.MolGraphConvFeaturizer(use_edges=True)
  elif feat == "gcn":
    featurizer = dc.feat.ConvMolFeaturizer()
  X = featurizer.featurize(smiles)
  dataset = dc.data.NumpyDataset(X=X, y=pki)
  # print(dataset)
  # print(pki)
  return dataset

In [8]:
# Metrics

metric = dc.metrics.Metric(
    dc.metrics.mean_absolute_error, task_averager=np.mean, mode="regression")

pearson_r2_score = dc.metrics.Metric(
    dc.metrics.pearson_r2_score, task_averager=np.mean, mode="regression")

mean_absolute_error = dc.metrics.Metric(
    dc.metrics.mean_absolute_error, task_averager=np.mean, mode="regression")

rms_score = dc.metrics.Metric(
    dc.metrics.rms_score, task_averager=np.mean, mode="regression")

### Trial GCNModel

In [ ]:
!pip install dgllife
# !pip install dgl
!pip install dgl-cu101 dglgo -f https://data.dgl.ai/wheels/repo.html

In [10]:
from deepchem.models.torch_models import AttentiveFPModel
from deepchem.models.wandblogger import WandbLogger

In [11]:
# trial model 



def train_attentiveFP(train, val, tid, num_layers=3, num_timesteps=2, graph_feat_size=200, num_epoch=5):


  callback = dc.models.ValidationCallback(val, 1000, metric)
  logger = WandbLogger(project='attentiveFP', name=tid)
  model = AttentiveFPModel(mode='regression', 
                           n_tasks=1, 
                           batch_size=16,
                    learning_rate=0.001, 
                    num_layers=num_layers, 
                    num_timesteps=num_timesteps,
                    graph_feat_size=graph_feat_size, 
                    wandb_logger=logger)
  loss = model.fit(train, nb_epoch=num_epoch, callbacks=callback)
  logger.finish()


  model_mae = model.evaluate(val, [mean_absolute_error], per_task_metrics=False)
  model_r2 = model.evaluate(val, [pearson_r2_score], per_task_metrics=False)
  model_rmse = model.evaluate(val, [rms_score], per_task_metrics=False)

  print(f"mae: {model_mae}")
  print(f"r2: {model_r2}")
  print(f"rmse: {model_rmse}")

  return model

In [20]:

from deepchem.models.torch_models import GATModel
def train_GAT(train, val, tid, num_epoch=5):


  callback = dc.models.ValidationCallback(val, 500, metric)
  logger = WandbLogger(project='gat', name=tid)
  model = GATModel(mode='regression', n_tasks=1, 
                    learning_rate=0.001, wandb_logger=logger)
  # model = GCNModel(mode='regression', n_tasks=1, batch_size=16, learning_rate=0.01, graph_conv_layers=[64]*num_layers)
  loss = model.fit(train, nb_epoch=num_epoch, callbacks=callback)
  logger.finish()


  model_mae = model.evaluate(val, [mean_absolute_error], per_task_metrics=False)
  model_r2 = model.evaluate(val, [pearson_r2_score], per_task_metrics=False)
  model_rmse = model.evaluate(val, [rms_score], per_task_metrics=False)

  print(f"mae: {model_mae}")
  print(f"r2: {model_r2}")
  print(f"rmse: {model_rmse}")

  return model
  # return model_mae['mean-mean_absolute_error'], model_r2['mean-pearson_r2_score'], model_rmse['mean-rms_score']


In [13]:
def train_cross_validation(k_folds_dataset, test, tid):
  for k_fold in k_folds_dataset:
    fold_val = k_fold[1]
    fold_train = k_fold[0]
    model = train_attentiveFP(fold_train, fold_val, tid, num_epoch=50)

    #predict on test 
    model_mae = model.evaluate(test, [mean_absolute_error], per_task_metrics=False)
    model_r2 = model.evaluate(test, [pearson_r2_score], per_task_metrics=False)
    model_rmse = model.evaluate(test, [rms_score], per_task_metrics=False)

    print(f"mae: {model_mae}")
    print(f"r2: {model_r2}")
    print(f"rmse: {model_rmse}")

In [14]:
# split dataset
def split_data(dataset):
  from deepchem.splits.splitters import RandomSplitter

  splitter = RandomSplitter()
  train, val, test = splitter.train_valid_test_split(
      dataset=dataset,
      log_every_n=100
  )
  return train, val, test


In [15]:
# split dataset into k-fold 
def split_data_k_fold(dataset, k_fold):
  from deepchem.splits.splitters import RandomSplitter

  splitter = RandomSplitter()
  train, test = splitter.train_test_split(
      dataset,
      log_every_n=100,
      seed=None
  )
  k_folds = splitter.k_fold_split(
      train,
      k=k_fold
  )

  return k_folds, test

In [16]:
dataset_ids = ['11', '15', '51', '72', '87', '100', '107', '108', '114', '121', '129', '130', '136', '137', '138', '155', '165', '176', '194', '252', '259', '278', '280', '10142', '10193', '10280', '10627', '11290', '12209', '12952', '19905']
len(dataset_ids)

31

In [ ]:
def basic_attentionFP(data):
  tid = 'tid' + data
  dataset = csv_to_feat(tid)
  train, val, test = split_data(dataset)
  model = train_attentiveFP(train, val, tid, num_layers=2, num_timesteps=3, graph_feat_size=250, num_epoch=40)
  model_mae = model.evaluate(test, [mean_absolute_error], per_task_metrics=False)
  model_r2 = model.evaluate(test, [pearson_r2_score], per_task_metrics=False)
  model_rmse = model.evaluate(test, [rms_score], per_task_metrics=False)

  print(f'model mae: {model_mae}')
  print(f'model r2: {model_r2}')
  print(f'model rmse: {model_rmse}')

  return model_mae, model_r2, model_rmse

In [ ]:
basic_attentionFP('15')

wandb: Currently logged in as: venustheplanet. Use `wandb login --relogin` to force relogin


Step 1000 validation: mean_absolute_error=0.569988
Step 2000 validation: mean_absolute_error=0.495569


eval/mean_absolute_error_(0),█▁
train/loss,▅▅▄█▄█▅▅▃▄▄▄▃▇▃▅▃▂▃▁▁▁▁▁
eval/mean_absolute_error_(0),0.49557
train/loss,0.14735


mae: {'mean-mean_absolute_error': 0.5282957849542654}
r2: {'mean-pearson_r2_score': 0.5474195566511867}
rmse: {'mean-rms_score': 0.7153431330375695}
model mae: {'mean-mean_absolute_error': 0.5875889861958681}
model r2: {'mean-pearson_r2_score': 0.4474935852855248}
model rmse: {'mean-rms_score': 0.7801159567830949}


({'mean-mean_absolute_error': 0.5875889861958681},
 {'mean-pearson_r2_score': 0.4474935852855248},
 {'mean-rms_score': 0.7801159567830949})

In [ ]:
basic_attentionFP('15')

Step 1000 validation: mean_absolute_error=0.598904
Step 2000 validation: mean_absolute_error=0.560731


eval/mean_absolute_error_(0),█▁
train/loss,█▇▆▅▃▄█▄██▄▄▄▆█▃▄▆▄▂▂▃▃▁
eval/mean_absolute_error_(0),0.56073
train/loss,0.22854


mae: {'mean-mean_absolute_error': 0.5354961592836122}
r2: {'mean-pearson_r2_score': 0.5583018652060876}
rmse: {'mean-rms_score': 0.6890538302159607}
model mae: {'mean-mean_absolute_error': 0.5065625044544287}
model r2: {'mean-pearson_r2_score': 0.6252016479237724}
model rmse: {'mean-rms_score': 0.6602670741392767}


({'mean-mean_absolute_error': 0.5065625044544287},
 {'mean-pearson_r2_score': 0.6252016479237724},
 {'mean-rms_score': 0.6602670741392767})

In [ ]:
def cross_val_attentionFP(data):
  tid = 'tid' + data
  dataset = csv_to_feat(tid)
  k_folds, test = split_data(dataset, 5)
  train_cross_validation(k_folds, test, tid)

In [ ]:
basic_attentionFP('15')

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


wandb: Currently logged in as: venustheplanet. Use `wandb login --relogin` to force relogin


Step 1000 validation: mean-mean_absolute_error=0.653737
Step 2000 validation: mean-mean_absolute_error=0.7131


eval/mean-mean_absolute_error_(0),▁█
train/loss,██▅▆▂▇▃▅▃▇▆▃▃▂▄▂▄▄▄▄▃▂▂▁
eval/mean-mean_absolute_error_(0),0.7131
train/loss,0.19755


...

eval/mean-mean_absolute_error_(0),▁█
train/loss,██▃▆▃▃▁▃▆▅▂▃▅▂▃▄▃▅▂▄▃▃▃▃
eval/mean-mean_absolute_error_(0),0.68837
train/loss,0.55013


mae: {'mean-mean_absolute_error': 0.5697559399356927}
r2: {'mean-pearson_r2_score': 0.47825999380027406}
rmse: {'mean-rms_score': 0.7301282006197134}
mae: {'mean-mean_absolute_error': 0.5775391140022936}
r2: {'mean-pearson_r2_score': 0.4363821502456793}
rmse: {'mean-rms_score': 0.7811631525459434}


Step 1000 validation: mean-mean_absolute_error=0.713177
Step 2000 validation: mean-mean_absolute_error=0.6729


eval/mean-mean_absolute_error_(0),█▁
train/loss,▆▃█▃▁▆▁▃█▄▅▄▂▃▆▄▂▄▃▅▄▃▂▃
eval/mean-mean_absolute_error_(0),0.6729
train/loss,0.53663


mae: {'mean-mean_absolute_error': 0.6868715973715843}
r2: {'mean-pearson_r2_score': 0.39005019840594474}
rmse: {'mean-rms_score': 0.8615435915906765}
mae: {'mean-mean_absolute_error': 0.5894390488752799}
r2: {'mean-pearson_r2_score': 0.4539137551506592}
rmse: {'mean-rms_score': 0.7747446554628591}


In [17]:
def basic_GAT(data):
  tid = 'tid' + data
  dataset = csv_to_feat(tid)
  train, val, test = split_data(dataset)
  train_GAT(train, val, tid, num_epoch=200)

In [ ]:
basic_GAT('15')

Step 500 validation: mean-mean_absolute_error=0.678864
Step 1000 validation: mean-mean_absolute_error=0.608276
Step 1500 validation: mean-mean_absolute_error=0.597452
Step 2000 validation: mean-mean_absolute_error=0.561257


eval/mean-mean_absolute_error_(0),█▄▃▁
train/loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/mean-mean_absolute_error_(0),0.56126
train/loss,0.18346


mae: {'mean-mean_absolute_error': 0.5612573955139283}
r2: {'mean-pearson_r2_score': 0.46322028795555903}
rmse: {'mean-rms_score': 0.7768238012635321}


In [ ]:
basic_mpnn('15')

Step 500 validation: mean-mean_absolute_error=0.52196


eval/mean-mean_absolute_error_(0),▁
train/loss,█▄▅▂▁
eval/mean-mean_absolute_error_(0),0.52196
train/loss,0.52016


mae: {'mean-mean_absolute_error': 0.5219599300729852}
r2: {'mean-pearson_r2_score': 0.3446379716891399}
rmse: {'mean-rms_score': 0.7012838740923604}


In [22]:
def run_gat(num_runs, datasets):
  # results = {}
  mean_results = {}

  for id in datasets:
    results = {}

    mae_list = []
    r2_list = []
    rmse_list = []

    tid = 'tid'+id
    dataset = csv_to_feat(tid)
    # train, val, test = split_data(dataset)
    
    for run in range(num_runs):
      train, val, test = split_data(dataset)
      model = train_GAT(train, val, tid, num_epoch=80)
      mae = model.evaluate(test, [mean_absolute_error], per_task_metrics=False)
      r2 = model.evaluate(test, [pearson_r2_score], per_task_metrics=False)
      rmse = model.evaluate(test, [rms_score], per_task_metrics=False)
      mae_list.append(mae['mean-mean_absolute_error'])
      r2_list.append(r2['mean-pearson_r2_score'])
      rmse_list.append(rmse['mean-rms_score'])
    
    results[f'{tid}_mae'] = mae_list
    results[f'{tid}_r2'] = r2_list
    results[f'{tid}_rmse'] = rmse_list

    mean_mae = np.mean(mae_list)
    mean_r2 = np.mean(r2_list)
    mean_rmse = np.mean(rmse_list)

    mean_results[f'{tid}_mean_mae'] = mean_mae
    mean_results[f'{tid}_mean_r2'] = mean_r2
    mean_results[f'{tid}_mean_rmse'] = mean_rmse


    print(f'mean mae for {num_runs} for {tid}: {mean_mae}')
    print(f'mean r2 for {num_runs} for {tid}: {mean_r2}')
    print(f'mean rmse for {num_runs} for {tid}: {mean_rmse}')
   
    results_df = pd.DataFrame(results)
    results_df.to_csv(f"{gat_dir}/{tid}_gat.csv")

  return results, mean_results

In [23]:
results_gat, mean_results_gat = run_gat(3, dataset_ids)

train/loss,█▁▁▁
train/loss,0.82287


mae: {'mean-mean_absolute_error': 0.7600282299010773}
r2: {'mean-pearson_r2_score': 0.40987362424629886}
rmse: {'mean-rms_score': 1.0289244842272094}


train/loss,█▁▁▁
train/loss,0.57272


mae: {'mean-mean_absolute_error': 0.8160985290597129}
r2: {'mean-pearson_r2_score': 0.4980698201041404}
rmse: {'mean-rms_score': 1.0416440091106463}


train/loss,█▁▁▁
train/loss,0.63986


...

mae: {'mean-mean_absolute_error': 0.6358494112143821}
r2: {'mean-pearson_r2_score': 0.31270909449860834}
rmse: {'mean-rms_score': 0.7774133828056485}


train/loss,█▁▁▁
train/loss,0.49565


mae: {'mean-mean_absolute_error': 0.7192760538741977}
r2: {'mean-pearson_r2_score': 0.4159889305301589}
rmse: {'mean-rms_score': 0.917817490312094}


train/loss,█▁▁▁
train/loss,0.39628


mae: {'mean-mean_absolute_error': 0.7270648808627526}
r2: {'mean-pearson_r2_score': 0.25098827824710435}
rmse: {'mean-rms_score': 0.9199016539894797}
mean mae for 3 for tid19905: 0.6318015703850585
mean r2 for 3 for tid19905: 0.35532971525667295
mean rmse for 3 for tid19905: 0.8567261735041152


In [ ]:
def run_attentiveFP(num_runs, datasets):
  # results = {}
  mean_results = {}

  for id in datasets:
    results = {}

    mae_list = []
    r2_list = []
    rmse_list = []

    tid = 'tid'+id
    dataset = csv_to_feat(tid)
    # train, val, test = split_data(dataset)
    
    for run in range(num_runs):
      train, val, test = split_data(dataset)
      model = train_attentiveFP(train, val, tid, num_layers=2, num_timesteps=3, graph_feat_size=250, num_epoch=40)
      mae = model.evaluate(test, [mean_absolute_error], per_task_metrics=False)
      r2 = model.evaluate(test, [pearson_r2_score], per_task_metrics=False)
      rmse = model.evaluate(test, [rms_score], per_task_metrics=False)
      mae_list.append(mae['mean-mean_absolute_error'])
      r2_list.append(r2['mean-pearson_r2_score'])
      rmse_list.append(rmse['mean-rms_score'])
    
    results[f'{tid}_mae'] = mae_list
    results[f'{tid}_r2'] = r2_list
    results[f'{tid}_rmse'] = rmse_list

    mean_mae = np.mean(mae_list)
    mean_r2 = np.mean(r2_list)
    mean_rmse = np.mean(rmse_list)

    mean_results[f'{tid}_mean_mae'] = mean_mae
    mean_results[f'{tid}_mean_r2'] = mean_r2
    mean_results[f'{tid}_mean_rmse'] = mean_rmse


    print(f'mean mae for {num_runs} for {tid}: {mean_mae}')
    print(f'mean r2 for {num_runs} for {tid}: {mean_r2}')
    print(f'mean rmse for {num_runs} for {tid}: {mean_rmse}')
   
    results_df = pd.DataFrame(results, index=[0])
    results_df.to_csv(f"{attentiveFP_dir}/{tid}_basic_mpnn.csv")

  return results, mean_results

# model_mae['mean-mean_absolute_error'], model_r2['mean-pearson_r2_score'], model_rmse['mean-rms_score']

In [ ]:
basic_gcn_results, mean_basic_gcn_results = run_attentiveFP(1, dataset_ids)

Step 1000 validation: mean_absolute_error=0.753964


eval/mean_absolute_error_(0),▁
train/loss,█▄▇▅▃▂▆▃▆▃▁▂▂
eval/mean_absolute_error_(0),0.75396
train/loss,0.48277


mae: {'mean-mean_absolute_error': 0.7356692785446713}
r2: {'mean-pearson_r2_score': 0.4556267521530057}
rmse: {'mean-rms_score': 0.9418799500819736}
mean mae for 1 for tid11: 0.745408190768107
mean r2 for 1 for tid11: 0.42588370863447245
mean rmse for 1 for tid11: 1.0294175218703174


Step 1000 validation: mean_absolute_error=0.616481
Step 2000 validation: mean_absolute_error=0.57218


eval/mean_absolute_error_(0),█▁
train/loss,▅▇▃██▆▄▃▃▂▁▁▄▁█▂█▃▂▃▃▂▆▂
eval/mean_absolute_error_(0),0.57218
train/loss,0.42832


...

eval/mean_absolute_error_(0),█▁
train/loss,▃█▂▃▃▄▃▂▂▃▃▂▁▃▃▃▂▂▂▂
eval/mean_absolute_error_(0),0.58833
train/loss,0.36192


mae: {'mean-mean_absolute_error': 0.5733005933230826}
r2: {'mean-pearson_r2_score': 0.5604703845314266}
rmse: {'mean-rms_score': 0.7595554169374789}
mean mae for 1 for tid12952: 0.5195177761373162
mean r2 for 1 for tid12952: 0.6458755966529663
mean rmse for 1 for tid12952: 0.643072221157002


Step 1000 validation: mean_absolute_error=0.681834


eval/mean_absolute_error_(0),▁
train/loss,▆▂▂▁█▄▂▃▅▂▁▂▁▃
eval/mean_absolute_error_(0),0.68183
train/loss,0.57386


mae: {'mean-mean_absolute_error': 0.623786303241742}
r2: {'mean-pearson_r2_score': 0.32411454421214886}
rmse: {'mean-rms_score': 0.7687733972494651}
mean mae for 1 for tid19905: 0.5178511125976171
mean r2 for 1 for tid19905: 0.43818886015950365
mean rmse for 1 for tid19905: 0.648481313826224


### Hyperparameter

In [ ]:
# Hyperparameters tuning
!pip install pyGPGO
import pyGPGO
from pyGPGO.covfunc import matern32
from pyGPGO.acquisition import Acquisition
from pyGPGO.surrogates.GaussianProcess import GaussianProcess
from pyGPGO.GPGO import GPGO



In [ ]:
params_dict = {
'dense_layer_size': ('int', [16, 2048]),
'graph_conv_layer_size': ('int', [32, 2048]),
'dropout': ('cont', [.0 ,0.5]),
'batch_size': ('int', [10,100]),
'learning_rate': ('cont', [0.0001, 0.0020]),
'nb_epoch': ('int', [20, 200])
}

In [ ]:
df_param = pd.DataFrame.from_dict(params_dict,orient='index', )
display(df_param)

In [ ]:
from hyperopt import hp, fmin, tpe, Trials

In [ ]:
search_space = {
    # 'layer_sizes': hp.choice('layer_sizes',[[500], [1000], [2000],[1000,1000]]),
    # 'dropouts': hp.uniform('dropout',low=0.2, high=0.5),
    'num_layers': hp.uniform('num_layers',low=1, high=6),
    'num_timesteps': hp.uniform('num_timesteps', low=1, high=10),
    'graph_feat_size': hp.uniform('graph_feat_size', low=50, high=400),
    'dropout': hp.uniform('dropout', low=0.0, high=0.8),
    # 'num_timesteps': hp.uniform('num_timesteps', low=1, high=10),
}

In [ ]:
"""
  callback = dc.models.ValidationCallback(val, 500, metric)
  logger = WandbLogger(project='gat', name=tid)
  model = GATModel(mode='regression', n_tasks=1, 
                    learning_rate=0.001, wandb_logger=logger)
  # model = GCNModel(mode='regression', n_tasks=1, batch_size=16, learning_rate=0.01, graph_conv_layers=[64]*num_layers)
  loss = model.fit(train, nb_epoch=num_epoch, callbacks=callback)
  logger.finish()


  model_mae = model.evaluate(val, [mean_absolute_error], per_task_metrics=False)
  model_r2 = model.evaluate(val, [pearson_r2_score], per_task_metrics=False)
  model_rmse = model.evaluate(val, [rms_score], per_task_metrics=False)

  print(f"mae: {model_mae}")
  print(f"r2: {model_r2}")
  print(f"rmse: {model_rmse}")

  return model_mae['mean-mean_absolute_error'], model_r2['mean-pearson_r2_score'], model_rmse['mean-rms_score']
"""

In [ ]:
import tempfile
#tempfile is used to save the best checkpoint later in the program.

metric = dc.metrics.Metric(
    dc.metrics.mean_absolute_error, mode="regression")
    # dc.metrics.mean_absolute_error, task_averager=np.mean, mode="regression")

def fm(args):
  save_dir = tempfile.mkdtemp()
  model = AttentiveFPModel(n_tasks=1,mode='regression',
                           num_layers=int(round(args['num_layers'])), 
                           num_timesteps=int(round(args['num_timesteps'])))
  #validation callback that saves the best checkpoint, i.e the one with the maximum score.
  validation=dc.models.ValidationCallback(val, 100, [metric],save_dir=save_dir,save_on_minimum=False)
  
  model.fit(train, nb_epoch=30,callbacks=validation)

  #restoring the best checkpoint and passing the negative of its validation score to be minimized.
  model.restore(model_dir=save_dir)
  valid_score = model.evaluate(val, [metric], per_task_metrics=False)
  # valid_score = model.evaluate(val, [mean_absolute_error], per_task_metrics=False)

  return valid_score['mean_absolute_error']
  # return valid_score['mean-mean_absolute_error']

In [ ]:
dataset = csv_to_feat('tid15')
train, test, val = split_data(dataset)

In [ ]:
trials=Trials()
best = fmin(fm,
    		space= search_space,
    		algo=tpe.suggest,
    		max_evals=15,
    		trials = trials)

  0%|          | 0/15 [00:00<?, ?it/s, best loss: ?]Step 100 validation: mean_absolute_error=0.842161
Step 200 validation: mean_absolute_error=0.75254
Step 300 validation: mean_absolute_error=0.699526
  7%|▋         | 1/15 [03:16<45:56, 196.86s/it, best loss: 0.8421605133110799]Step 100 validation: mean_absolute_error=0.862421
Step 200 validation: mean_absolute_error=0.840221
Step 300 validation: mean_absolute_error=0.716911
 13%|█▎        | 2/15 [07:16<48:08, 222.18s/it, best loss: 0.8421605133110799]Step 100 validation: mean_absolute_error=0.843041
Step 200 validation: mean_absolute_error=0.76967
Step 300 validation: mean_absolute_error=0.734506
 20%|██        | 3/15 [09:50<38:10, 190.89s/it, best loss: 0.8421605133110799]Step 100 validation: mean_absolute_error=0.844787
Step 200 validation: mean_absolute_error=0.701256
Step 300 validation: mean_absolute_error=0.710181
 27%|██▋       | 4/15 [13:45<38:10, 208.27s/it, best loss: 0.8421605133110799]Step 100 validation: mean_absolute_err

In [ ]:
print(best)

{'dropout': 0.11128726856892622, 'graph_feat_size': 250.23315502596134, 'num_layers': 1.059144167503233, 'num_timesteps': 2.9582410849143295}


In [ ]:
int(round(2.8025094780246227))

3

In [ ]:
def hyper_model_attentiveFP(dense_layer_size, graph_conv_layer_size, batch_size, dropout, nb_epoch, **params):
     
    global neg_mae
    global mae
    global r2
    global rmse
    global config
    
    dense_layer_size = int(round(dense_layer_size))
    graph_conv_depth = 2 # two graph convolutional layers
    graph_conv_layer_size = int(round(graph_conv_layer_size))
    batch_size = int(round(batch_size))
    dropout = dropout
    model = dc.models.GraphConvModel(
            len(tasks),
            batch_size=batch_size,
            dropout=dropout,
            mode='regression',
            graph_conv_layers=[graph_conv_layer_size]*graph_conv_depth,
            dense_layer_size=dense_layer_size,
            # configproto=config,
            **params
            )
    model.fit(train_gcn)

    # MAE
    valid_scores_mae = model.evaluate(val_gcn, [mean_absolute_error], per_task_metrics=False)
    # R2
    valid_scores_r2 = model.evaluate(val_gcn, [pearson_r2_score], per_task_metrics=False)
    # RMSE
    valid_scores_rmse = model.evaluate(val_gcn, [rms_score], per_task_metrics=False)
    # -MAE
    neg_valid_scores_mae = -valid_scores_mae['mean-mean_absolute_error']

    neg_mae.append(neg_valid_scores_mae)
    mae.append(valid_scores_mae['mean-mean_absolute_error'])
    r2.append(valid_scores_r2['mean-pearson_r2_score'])
    rmse.append(valid_scores_rmse['mean-rms_score'])
    
    return neg_valid_scores_mae

In [ ]:
# config = tf.compat.v1.ConfigProto()
# config = tf.ConfigProto()
# config.gpu_options.allow_growth = True

# Run the model 

neg_mae = []
mae = []
r2  = []
rmse = []
cov = matern32()
gp  = GaussianProcess(cov)
acq = Acquisition(mode='ExpectedImprovement')
tasks = ['pki']

gpgo = GPGO(gp, acq, hyper_model, params_dict)

# Adjust the number of iterations, e.g., max_iter=100 and init_eval=10.
max_iter = 10
init_evals = 1
gpgo.run(max_iter=max_iter, init_evals=init_evals)

In [ ]:
# hyperparameter tuning
"""
from deepchem.models import GCNModel

def hyper_model(dense_layer_size, graph_conv_layer_size, batch_size, dropout, nb_epoch, **params):
     
    global neg_mae
    global mae
    global r2
    global rmse
    global config
    
    dense_layer_size = int(round(dense_layer_size))
    graph_conv_depth = 2 # two graph convolutional layers
    graph_conv_layer_size = int(round(graph_conv_layer_size))
    batch_size = int(round(batch_size))
    dropout = dropout
    model = GCNModel(
            n_tasks=1,
            graph_conv_layers=[graph_conv_layer_size]*graph_conv_depth,
            batch_size=batch_size,
            dropout=dropout,
            mode='regression',
            dense_layer_size=dense_layer_size,
            configproto=config,
            **params
            )
    model.fit(train)

    # MAE
    valid_scores_mae = model.evaluate(val, [mean_absolute_error], per_task_metrics=False)
    # R2
    valid_scores_r2 = model.evaluate(val, [pearson_r2_score], per_task_metrics=False)
    # RMSE
    valid_scores_rmse = model.evaluate(val, [rms_score], per_task_metrics=False)
    # -MAE
    neg_valid_scores_mae = -valid_scores_mae['mean-mean_absolute_error']

    neg_mae.append(neg_valid_scores_mae)
    mae.append(valid_scores_mae['mean-mean_absolute_error'])
    r2.append(valid_scores_r2['mean-pearson_r2_score'])
    rmse.append(valid_scores_rmse['mean-rms_score'])
    
    return neg_valid_scores_mae
"""